In [ ]:
import pandas as pd
from rapidfuzz import process, fuzz
import src as src
pd.set_option('display.max_columns', None)

In [ ]:
df_eventos = pd.read_csv("data/df_eventos_liga2015.csv")

In [ ]:
df_jugadores_info = pd.read_csv("data/jugadores_info.csv")

In [ ]:
df_jugadores_info[df_jugadores_info["playerFullName"] == "Sergio Álvarez"]

In [ ]:
df_jugadores_info[df_jugadores_info["playerFullName"] == "Pablo Hernández"]

In [ ]:
df_jugadores_info[df_jugadores_info["playerFullName"] == "Pedro López"]

In [ ]:
# Cambiar el primer 'Sergio Alvarez'
sergio_idx = df_jugadores_info[df_jugadores_info['playerFullName'] == 'Sergio Álvarez'].index[0]
df_jugadores_info.at[sergio_idx, 'playerFullName'] = 'Sergio Álvarez Conde'

In [ ]:
# Cambiar el primer 'Pablo Hernandez'
pablo_idx = df_jugadores_info[df_jugadores_info['playerFullName'] == 'Pablo Hernández'].index[0]
df_jugadores_info.at[pablo_idx, 'playerFullName'] = 'Pedro Pablo Hernández'

In [ ]:
# Cambiar el primer 'Pedro López'
pedro_idx = df_jugadores_info[df_jugadores_info['playerFullName'] == 'Pedro López'].index[1]
df_jugadores_info.at[pedro_idx, 'playerFullName'] = 'Pedro López Muñoz'

In [ ]:
df_jugadoresunicos_eventos = df_eventos["player"].unique().tolist()
df_jugadoresunicos_info = df_jugadores_info["playerFullName"].unique().tolist()
df_clubesunicos_eventos = df_eventos["team"].unique().tolist()
df_clubesunicos_info = df_jugadores_info["club"].unique().tolist()

In [ ]:
df_nombres = src.normalizacion_nombres(df_jugadoresunicos_info, df_jugadoresunicos_eventos)

In [ ]:
df_clubes = src.normalizacion_nombres(df_clubesunicos_info, df_clubesunicos_eventos)

In [ ]:
# Crear diccionario para mapear los nombres
dict_nombres_info = df_nombres.set_index('Nombre_original')['Nombre_unificado'].to_dict()
# Mapear nombres jugadores
df_jugadores_info["playerFullName"] = df_jugadores_info["playerFullName"].apply(
    lambda x: dict_nombres_info.get(x, x) if dict_nombres_info.get(x) is not None else x)

In [ ]:
# Crear diccionario para mapear los equipos
df_equipos_jugadores = df_eventos[["player", "team"]].drop_duplicates()
dict_equipos_jugadores = dict(zip(df_equipos_jugadores["player"], df_equipos_jugadores["team"]))
# Mapear los equipos en la temporada 2015/2016 ya que la info que aparece es actual
df_jugadores_info["club"] = df_jugadores_info["playerFullName"].map(dict_equipos_jugadores)

In [ ]:
# Crear diccionarios para mapear los IDs de jugador y equipo
df_equipos = df_eventos[["team", "team_id"]].drop_duplicates()
dict_equipos = dict(zip(df_equipos["team"], df_equipos["team_id"]))
df_jugadores = df_eventos[["player", "player_id"]].drop_duplicates()
dict_jugadores = dict(zip(df_jugadores["player"], df_jugadores["player_id"]))

In [ ]:
# Mapear ID Jugadores
df_jugadores_info["playerID"] = df_jugadores_info["playerFullName"].map(dict_jugadores)
# Mapear ID Equipos
df_jugadores_info["clubID"] = df_jugadores_info["club"].map(dict_equipos)

In [2]:
df_jugadores_info = pd.read_csv("data/jugadores_LaLiga_completo_normalizado.csv")

In [3]:
df_tabla = pd.read_csv("data/jugadores_info_normalizado.csv")

In [ ]:
df_tabla.columns

In [4]:
df_jugadores_info.drop(columns=(['player_name', 'team_id', 'player_country_code', 'age',
       'positions', 'unused_sub',
       'team_gls_on_pitch', 'team_gls_ag_on_pitch', 'per90_plus_minus',
       'per90_on_off']), inplace=True)

In [5]:
df_combinado = df_tabla.merge(
    df_jugadores_info,
    left_on=["playerID"],
    right_on=["player_id"],
    how="left"
)

In [8]:
df_combinado.drop(columns=(["player_id"]), inplace=True)

In [9]:
df_combinado.to_csv("data/jugadores_info_normalizado.csv", index=False)